# Generacion de avatares para la Lengua de Señas Cubana


## Importar bibliotecas

In [ ]:
# !pip install mediapipe


In [ ]:
# !pip install scipy --upgrade --user


In [ ]:
# !pip3 install --upgrade pip setuptools wheel


In [ ]:
# !sudo pip3 install - -upgrade pip setuptools wheel


In [ ]:
# !pip install future_fstrings wheel


In [ ]:
# !sudo apt install subversion


In [ ]:
# !pip install bpy


In [ ]:
# !pip install bpy_post_install


In [ ]:
import cv2
import mediapipe as mp


In [ ]:

# import bpy


In [ ]:
from typing import List

from numpy import ndarray

from scipy.interpolate import RegularGridInterpolator, interp1d
import json
import os
import math


## Instanciar datos

## Animaciones 

In [ ]:
# @markdown ### Importar bibliotecas
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib import rc
import numpy as np
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
from pathlib import Path


In [ ]:
def find_similar(word, vocab):
    import difflib
    best_match = difflib.get_close_matches(word, vocab)
    if len(best_match) != 0:
        score = difflib.SequenceMatcher(None, word, best_match[0]).ratio()
        return best_match[0], score
    else:
        return None


## Clases

In [ ]:
from scipy.spatial.transform import Rotation as R


def rotate(data, radian_x=0, radian_y=0, radian_z=0):
    r = R.from_rotvec(np.array([radian_x, radian_y, radian_z]))
    return r.apply(data)


def translate(data, dv):
    return data + dv


def get_angle_between_vectors(v1, v2, complement_angle=False):

    v1, v2 = v1 / np.linalg.norm(v1), v2 / np.linalg.norm(v2)

    prev_angle_cos = np.dot(v1, v2)
    prev_angle_sin = np.linalg.norm(np.cross(v1, v2))

    angle = np.arctan2(prev_angle_sin, prev_angle_cos)

    if complement_angle:
        angle = 2 * np.pi - angle

    return angle


def cross_product(v1, v2):
    return np.cross(v1, v2)


def rodriquez_rotation_formula(v, angle, axis):
    axis = axis / np.linalg.norm(axis)
    return v * np.cos(angle) + cross_product(axis, v) * np.sin(angle) + axis * np.dot(axis, v) * (1 - np.cos(angle))


def rodriquez_rotation_from_point(base, v, angle, axis):
    v_prime = v - base
    v_prime = rodriquez_rotation_formula(v_prime, angle, axis)
    return v_prime + base


def get_axis_rotation_angle(v1):
    """
    return XY plane rotation around Z axis, YZ plane rotation around X axis
    """
    rotations = []

    # XY plane angle from X axis
    v = v1[[0, 1]]
    rotation = get_angle_between_vectors(np.array([1, 0]), v)
    rotations.append(rotation)

    # YZ plane angle from Y axis
    v = v1[[1, 2]]
    rotation = get_angle_between_vectors(np.array([1, 0]), v)
    rotations.append(rotation)

    # XZ plane angle from X axis
    v = v1[[0, 2]]
    rotation = get_angle_between_vectors(np.array([1, 0]), v)
    rotations.append(rotation)

    return rotations


def get_rotation_align_matrix(vec1, vec2):
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 /
                                                      np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    if np.any(v):
        c = np.dot(a, b)
        s = np.linalg.norm(v)
        kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1-c) / (s**2))
        return rotation_matrix
    else:
        return np.eye(3)


In [ ]:
import os
from pathlib import Path


class BvhNode(object):
    def __init__(
        self, name, offset, rotation_order,
        children=None, parent=None, is_root=False, is_end_site=False
    ):
        if not is_end_site and \
                rotation_order not in ['xyz', 'xzy', 'yxz', 'yzx', 'zxy', 'zyx']:
            raise ValueError(f'Rotation order invalid.')
        self.name = name
        self.offset = offset
        self.rotation_order = rotation_order
        self.children = children
        self.parent = parent
        self.is_root = is_root
        self.is_end_site = is_end_site


class BvhHeader(object):
    def __init__(self, root, nodes):
        self.root = root
        self.nodes = nodes


def write_header(writer, node, level):
    indent = ' ' * 4 * level
    if node.is_root:
        writer.write(f'{indent}ROOT {node.name}\n')
        channel_num = 6
    elif node.is_end_site:
        writer.write(f'{indent}End Site\n')
        channel_num = 0
    else:
        writer.write(f'{indent}JOINT {node.name}\n')
        channel_num = 3
    writer.write(f'{indent}{"{"}\n')

    indent = ' ' * 4 * (level + 1)
    writer.write(
        f'{indent}OFFSET '
        f'{node.offset[0]} {node.offset[1]} {node.offset[2]}\n'
    )
    if channel_num:
        channel_line = f'{indent}CHANNELS {channel_num} '
        if node.is_root:
            channel_line += f'Xposition Yposition Zposition '
        channel_line += ' '.join([
            f'{axis.upper()}rotation'
            for axis in node.rotation_order
        ])
        writer.write(channel_line + '\n')

    for child in node.children:
        write_header(writer, child, level + 1)

    indent = ' ' * 4 * level
    writer.write(f'{indent}{"}"}\n')


def write_bvh(output_file, header, channels, frame_rate=30):
    output_file = Path(output_file)
    if not output_file.parent.exists():
        os.makedirs(output_file.parent)

    with output_file.open('w') as f:
        f.write('HIERARCHY\n')
        write_header(writer=f, node=header.root, level=0)

        f.write('MOTION\n')
        f.write(f'Frames: {len(channels)}\n')
        f.write(f'Frame Time: {1 / frame_rate}\n')

        for channel in channels:
            f.write(' '.join([f'{element}' for element in channel]) + '\n')


In [ ]:
from enum import Enum


class TypeOfBodyPart(Enum):
    BODY = 0
    LHAND = 1
    RHAND = 2


In [ ]:
class Bodypart:
    def __init__(self, name: str, type: TypeOfBodyPart, points: ndarray):
        self.name = name
        self.type = type
        self.points = points
        self.shape = points.shape
        self.isZero = (points == 0.0).all()

    def normalize(self):
        pass

    def plot(self):
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')

        ax.scatter(self.points[:, 0], self.points[:, 1], self.points[:, 2])

        base_points = [0, 5, 9, 13, 17]

        ax.scatter(self.points[base_points, 0],
                   self.points[base_points, 1], self.points[base_points, 2])

        ax.set_xlabel('X Label')
        ax.set_ylabel('Y Label')
        ax.set_zlabel('Z Label')

        plt.show()


class DefaultRightHand(Bodypart):
    def __init__(self):
        super().__init__("Default Right Hand", TypeOfBodyPart.RHAND, np.array([
            [4.32893813e-01,  7.13910162e-01,  1.74915698e-07],
            [4.47495788e-01,  6.83327675e-01, -3.24304728e-03],
            [4.72318083e-01,  6.60035312e-01, -7.41990516e-03],
            [4.96919394e-01,  6.48758709e-01, -1.07117742e-02],
            [5.17513335e-01,  6.45120084e-01, -1.34482430e-02],
            [4.72233117e-01,  6.68168843e-01, -1.39389271e-02],
            [5.04080117e-01,  6.68054163e-01, -2.02744883e-02],
            [5.20316601e-01,  6.73237920e-01, -2.24908795e-02],
            [5.30374765e-01,  6.79046035e-01, -2.31099464e-02],
            [4.72281843e-01,  6.94146156e-01, -1.56589523e-02],
            [5.06882370e-01,  6.95890546e-01, -2.05966737e-02],
            [5.24215281e-01,  6.98764563e-01, -2.14019585e-02],
            [5.34511805e-01,  6.99780941e-01, -2.19137203e-02],
            [4.71816361e-01,  7.19911575e-01, -1.70433484e-02],
            [5.05639672e-01,  7.22618699e-01, -2.10257601e-02],
            [5.22720635e-01,  7.23885953e-01, -2.05061063e-02],
            [5.33262134e-01,  7.21632302e-01, -2.00466551e-02],
            [4.71493304e-01,  7.43003249e-01, -1.82789769e-02],
            [4.99223262e-01,  7.46479690e-01, -2.12228913e-02],
            [5.13656974e-01,  7.46875286e-01, -2.01588757e-02],
            [5.24085701e-01,  7.43879616e-01, -1.89451054e-02],]))


class DefaultLeftHand(Bodypart):
    def __init__(self):

        rhand = DefaultRightHand().points
        mirror_x_matrix = np.array([
            [-1, 0, 0],
            [0, 1, 0],
            [0, 0, 1],
        ])

        lhand = rhand @ mirror_x_matrix

        super().__init__("Default Left Hand", TypeOfBodyPart.LHAND, lhand)


# Explicaciones

## Crop movement

Este problema aparece cuando en los extremos de los videos la persona
ejecutante se mantiene en una misma posición. Este acto es natural en enseñanza
pero antinatural en conversaciones, por lo tanto se debe de eliminar los frames 
en donde ocurra. 

### Función de movimiento

Se define m como la función de movimiento, donde f devuelve el frame en el tiempo

m(t) = |f(t) - f(t-1)|

m cuantifica el movimiento realizado entre frame y frame, por lo tanto se puede decir
que describe la cantidad de movimiento en el video. Si se integra esta funci'on se obtiene
una medida de la cantidad de movimiento en un intervalo.

### Integral de funcion de movimiento

Se desea quedarse con el intervalo que contenga la mayor cantidad de movimiento cumpliendo
ciertas restricciones. 
- El intervalo no debe de ser completo: Por lo tanto a mayor intervalo mayor debe ser la penalizacion
de escogerlo
- El intervalo debe contener la mayor cantidad de movimiento posible: Por lo tanto a mayor intervalo
mayor es el beneficio por escogerlo
- Se necesita que gran parte del movimiento este presente en el intervalo: Por lo tanto la cantidad total
del intervalo debe superar el 80%

Se selecciona la funcion interv, la cual dado i y j con i < j, devuelve el beneficio de seleccionar el intervalo.
Y se desea maximizar este beneficio.

max i,j tal que => interv(i,j) = integral(m)(i,j) / (j - i)

sujeto a que integral(m)(i,j)/TotalDeMovimiento >= 0.8

Esta funcion se le encontraron los i y j mediante una busqueda de entre todos
los intervalos validos.

La funcion integral se aproxima mediante metodos numericos con los datos discretos 
obtenidos de la funcion m.

Al la intergral ser discreta, su aproximacion es calculada con una sumatoria.

## Normalización

Para la normalizacion existen varios problemas: 
- Manos inexistentes en el medio: Se interpola el hueco
- Manos inexistentes al inicio o final: No se puede interpolar pq no se tiene llegada o partida

### Rellenar inicio y final con default

Se extrajo una configuracion de puntos pertenecientes a una mano es reposo y esta
fue utilizada para la creacion de una mano por defecto para rellenar. Se extrajo 
la mano derecha y esta se reflejo sobre un eje de coordenadas para obtener la mano izquierda

La mano fue escalada para que se asemejara a las dimensiones del avatar en la que se va a poner

El problema ahora consiste en ubicar correctamente la mano. En primer lugar 
se encontro el angulo entre el vector del codo y la muñeca con el del la 
base de la mano y el punto medio de la base del meñique con el indice. La 
mano fue rotada sobre el producto cruz de ambos vectores el angulo establecido y
posicionada de tal forma que la base de la mano coincida con la muñeca del cuerpo.

### Rellenar huecos intermedios

Estos huecos fueron llenados on interpolacion entre los puntos en donde se terminaba el segmento
y los puntos donde continuaba. Para proveer la cantidad de puntos necesarios para el procedimiento
se seleccionaban de ser posible 2 frame de la seccion terminante y dos frame de la seccion continuante.
Se puede realizar varios tipos de interpolación, lineal, cúbica.

## Concatenación de tokens

Para la concatenación de tokens se realiza mediante la interpolación. El problema aquí es 
que existen poses que terminan más distantes de su continuación que otras, haciendo que 
la transición sea o muy rápida o muy lenta, por esto se decide hacer una transición con frames
variables en dependencia de la diferencia.

### Frames variables intermedios

A criterio del autor se seleccionaron unos ejemplos en donde se sabia la variabilidad y la cantidad de frames
buena para el segmento.

Variacion           0 0.5 4.6
Cantidad de frames  1   7  14

con estos datos se interpoló linealmente una función para que sirviera para seleccionar la cantidad de frames de 
manera automática.






In [ ]:
class Skeleton:

    default_lhand = DefaultLeftHand()

    default_rhand = DefaultRightHand()

    def __init__(self, body: Bodypart, lhand: Bodypart, rhand: Bodypart):
        self.body = body  # 25 points
        self.lhand = lhand  # 21 points
        self.rhand = rhand  # 21 points

        self.body_dim = 25*3
        self.left_dim = 21*3
        self.right_dim = 21*3

        self.dims = [(25, 3), (21, 3), (21, 3)]

        self.bodyjoints = [[1, 7], [4, 8], [9, 10], [11, 12], [11, 13], [
            12, 14], [11, 23], [12, 24], [23, 24], [13, 15], [14, 16]]

        self.leftjoints = [[0, 1], [1, 2], [2, 3], [3, 4],
                           [0, 5], [5, 6], [6, 7], [7, 8],
                           [5, 9], [9, 10], [10, 11], [11, 12],
                           [9, 13], [13, 14], [14, 15], [15, 16],
                           [13, 17], [17, 18], [18, 19], [19, 20], [0, 17]]

        self.rightjoints = self.leftjoints

        self.fullbodyjoints = [self.bodyjoints,
                               self.leftjoints, self.rightjoints]

        self.parts_names = ["body", "left", "right"]

    @property
    def bodyparts(self):
        return [self.body, self.lhand, self.rhand]

    @property
    def all_points(self):
        points = np.concatenate([x.points for x in self.bodyparts])
        return points

    def get_position_default_hand(self, is_left: bool, hand_forearm_proportion=1/11.15, finger_proportion=1/0.68):
        if is_left:
            elbow, wrist, body_pinky, body_index, body_thumb = self.body.points[[
                13, 15, 17, 19, 21]]
            hand = DefaultLeftHand()
        else:
            elbow, wrist, body_pinky, body_index, body_thumb = self.body.points[[
                14, 16, 18, 20, 22]]
            hand = DefaultRightHand()

        finger_body_mean = (body_pinky + body_index + body_thumb) / 3
        arm_director_vector = wrist - elbow

        base_hand, index_finger, pinky_finger = hand.points[[0, 5, 17]]
        knuckles_hand = (index_finger + pinky_finger) / 2

        hand_director_vector = knuckles_hand - base_hand

        # Normalize hand
        factor = hand_forearm_proportion * \
            np.linalg.norm(arm_director_vector) / \
            np.linalg.norm(hand_director_vector)
        factor2 = finger_proportion * \
            np.linalg.norm(body_pinky - body_index) / \
            np.linalg.norm(pinky_finger - index_finger)
        hand_points = hand.points.copy()
        hand_points[:, 0] = hand_points[:, 0] * factor
        hand_points[:, 1] = hand_points[:, 1] * factor2

        # Recalculate points with normalized hand
        base_hand, index_finger, pinky_finger = hand_points[[0, 5, 17]]
        knuckles_hand = (index_finger + pinky_finger) / 2
        hand_director_vector = knuckles_hand - base_hand

        # Translate hand to have 0 at base_hand
        hand_points = translate(hand_points, - base_hand)

        a = get_angle_between_vectors(
            hand_director_vector, arm_director_vector)
        compl_a = get_angle_between_vectors(
            hand_director_vector, arm_director_vector, True)
        # a_180 = a + np.pi

        axis = np.cross(hand_director_vector, arm_director_vector)

        def process_angle(angle):
            final_hand_points = np.array(
                [rodriquez_rotation_formula(v, angle, axis) for v in hand_points])
            final_hand_points = translate(final_hand_points, base_hand)
            hand_distance = np.mean(
                [np.linalg.norm(finger_body_mean - x) for x in final_hand_points])

            return final_hand_points, hand_distance

        hand_points_normal, hand_distance_normal = process_angle(a)
        hand_points_compl, hand_distance_compl = process_angle(compl_a)
        # hand_points_180, hand_distance_180 = process_angle(a_180)

        min_dist = min([
            hand_distance_compl,
            # hand_distance_180,
            hand_distance_normal
        ])

        if min_dist == hand_distance_normal:
            hand_points = hand_points_normal
        elif min_dist == hand_distance_compl:
            hand_points = hand_points_compl
        # elif min_dist == hand_distance_180:
        #     hand_points = hand_points_180

        return translate(hand_points, wrist - base_hand)

    def plot(self, figsize=(5.0, 5.0), elev=90, angle=1, dims=None, text=False, joints=True):
        if not dims:
            dims = self.dims

        skeletons_points = np.array([self.all_points])
        rc('animation', html='jshtml')
        fig_size_x, fig_size_y = figsize

        plt.rcParams["figure.figsize"] = [fig_size_x, fig_size_y]
        plt.rcParams['figure.autolayout'] = True

        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        colors = ['b', 'y', 'r', 'k']

        D = self._transform_data(skeletons_points)

        s = 0
        e = 0
        for j, dim in enumerate(dims):
            dim = dim[0]
            e += dim

            ax.plot(D[s:e, 0], D[s:e, 1], D[s:e, 2], f'{colors[j]}.')
            if text:
                for p in range(s, e):
                    ax.text3D(D[p, 0], D[p, 1], D[p, 2], str(p))
            if joints:
                for h, k in self.fullbodyjoints[j]:
                    ax.plot([D[s+h, 0], D[s+k, 0]], [D[s+h, 1], D[s+k, 1]],
                            [D[s+h, 2], D[s+k, 2]], f'{colors[j]}')
            s += dim

            # ax.set_axis_off()
            if elev and angle:
                ax.view_init(elev, angle)
        
        plt.show()

    def _transform_data(self, Data, i=0):
        from scipy.spatial.transform import Rotation as R

        r = R.from_rotvec(np.pi/2 * np.array([0, 0, 1]))

        X = np.array(Data[i, :, 0])
        Y = np.array(-Data[i, :, 1])
        Z = np.array(Data[i, :, 2])

        D = np.array([x for x in zip(X, Y, Z)])
        D = r.apply(D)
        return D
    
    
    def normalize(self):
        def translate_hand(hand, wrist):
            base_hand = hand.points[0]
            hand.points = translate(hand.points, wrist - base_hand)

        if not self.lhand.isZero:
            lwrist = self.body.points[15]
            translate_hand(self.lhand, lwrist)
        if not self.rhand.isZero:
            rwrist = self.body.points[16]
            translate_hand(self.rhand, rwrist)



In [ ]:
class TokenLSC:
    def __init__(self, name: str, sign: List[Skeleton], with_normalize=True, with_crop=True) -> None:
        self.name = name
        self.sign = sign
        self.fps = 24
        self.body_dim = 25*3
        self.left_dim = 21*3
        self.right_dim = 21*3

        self.dims = [(25, 3), (21, 3), (21, 3)]

        self.bodyjoints = [[1, 7], [4, 8], [9, 10], [11, 12], [11, 13], [
            12, 14], [11, 23], [12, 24], [23, 24], [13, 15], [14, 16]]

        self.leftjoints = [[0, 1], [1, 2], [2, 3], [3, 4],
                           [0, 5], [5, 6], [6, 7], [7, 8],
                           [5, 9], [9, 10], [10, 11], [11, 12],
                           [9, 13], [13, 14], [14, 15], [15, 16],
                           [13, 17], [17, 18], [18, 19], [19, 20], [0, 17]]

        self.rightjoints = self.leftjoints

        self.fullbodyjoints = [self.bodyjoints,
                               self.leftjoints, self.rightjoints]

        self.parts_names = ["body", "left", "right"]

        self.isNormalized = False

        self.isCropped = False

        if with_normalize:
            self.normalize()
        if with_crop:
            self.crop_movement()

        ##############################################################
        # BVH DATA
        #############################################################

        self.poses3d = np.array([sk.all_points for sk in self.sign]).reshape((len(self.sign),self.dims[0][0]+self.dims[1][0]+self.dims[2][0],self.dims[0][1]))

        self.root = 'NOSE_ROOT'  # ARTIFICIAL MIDDLE HIP MUST BE CREATED
        self.body_points_names = [
            "NOSE_ROOT",
            "LEFT_EYE_INNER",
            "LEFT_EYE",
            "LEFT_EYE_OUTER",
            "RIGHT_EYE_INNER",
            "RIGHT_EYE",
            "RIGHT_EYE_OUTER",
            "LEFT_EAR",
            "RIGHT_EAR",
            "MOUTH_LEFT",
            "MOUTH_RIGHT",
            "LEFT_SHOULDER",
            "RIGHT_SHOULDER",
            "LEFT_ELBOW",
            "RIGHT_ELBOW",
            "LEFT_WRIST",
            "RIGHT_WRIST",
            "LEFT_PINKY",
            "RIGHT_PINKY",
            "LEFT_INDEX",
            "RIGHT_INDEX",
            "LEFT_THUMB",
            "RIGHT_THUMB",
            "LEFT_HIP",
            "RIGHT_HIP",
            "LEFT_HAND_WRIST",
            "LEFT_THUMB_MCP",
            "LEFT_THUMB_PIP",
            "LEFT_THUMB_DIP",
            "LEFT_THUMB_TIP_ENDPOINT",
            "LEFT_INDEX_FINGER_MCP",
            "LEFT_INDEX_FINGER_PIP",
            "LEFT_INDEX_FINGER_DIP",
            "LEFT_INDEX_FINGER_TIP_ENDPOINT",
            "LEFT_MIDDLE_FINGER_MCP",
            "LEFT_MIDDLE_FINGER_PIP",
            "LEFT_MIDDLE_FINGER_DIP",
            "LEFT_MIDDLE_FINGER_TIP_ENDPOINT",
            "LEFT_RING_FINGER_MCP",
            "LEFT_RING_FINGER_PIP",
            "LEFT_RING_FINGER_DIP",
            "LEFT_RING_FINGER_TIP_ENDPOINT",
            "LEFT_PINKY_MCP",
            "LEFT_PINKY_PIP",
            "LEFT_PINKY_DIP",
            "LEFT_PINKY_TIP_ENDPOINT",
            "RIGHT_HAND_WRIST",
            "RIGHT_THUMB_MCP",
            "RIGHT_THUMB_PIP",
            "RIGHT_THUMB_DIP",
            "RIGHT_THUMB_TIP_ENDPOINT",
            "RIGHT_INDEX_FINGER_MCP",
            "RIGHT_INDEX_FINGER_PIP",
            "RIGHT_INDEX_FINGER_DIP",
            "RIGHT_INDEX_FINGER_TIP_ENDPOINT",
            "RIGHT_MIDDLE_FINGER_MCP",
            "RIGHT_MIDDLE_FINGER_PIP",
            "RIGHT_MIDDLE_FINGER_DIP",
            "RIGHT_MIDDLE_FINGER_TIP_ENDPOINT",
            "RIGHT_RING_FINGER_MCP",
            "RIGHT_RING_FINGER_PIP",
            "RIGHT_RING_FINGER_DIP",
            "RIGHT_RING_FINGER_TIP_ENDPOINT",
            "RIGHT_PINKY_MCP",
            "RIGHT_PINKY_PIP",
            "RIGHT_PINKY_DIP",
            "RIGHT_PINKY_TIP_ENDPOINT",

            # #ARTIFICIAL POINTS TO BE CREATED
            # 'HIP',
            # 'SPINE',
            # 'THORAX',
            # 'NECK'
        ]

        self.keypoint2index = {name: i for i,
                               name in enumerate(self.body_points_names)}
        self.index2keypoint = {v: k for k, v in self.keypoint2index.items()}
        self.keypoint_num = len(self.keypoint2index)
# Landmark connections (create bones out of two landmarks)
        body_points_connections = [
            ("LEFT_SHOULDER", "LEFT_ELBOW"),
            ("LEFT_ELBOW", "LEFT_WRIST"),
            ("RIGHT_SHOULDER", "RIGHT_ELBOW"),
            ("RIGHT_ELBOW", "RIGHT_WRIST"),

            ("RIGHT_WRIST", "RIGHT_INDEX"),  # hand bone right
            ("LEFT_WRIST", "LEFT_INDEX"),  # hand bone left

            # for left collar (invert all axes)
            ("LEFT_SHOULDER", "RIGHT_SHOULDER"),
            # for right collar (invert all axes)
            ("RIGHT_SHOULDER", "LEFT_SHOULDER"),

            # below jaw to lip upper middle to simulate neck bone
            ("152", "0", "NECK"),
            # lip upper middle to mid nose bridge to simulate head bone
            ("0", "6", "HEAD"),
            ("152", "10", "HEAD_NECK")  # below jaw to highest face point
        ]
        self.children = {
            # 'HIP': ['RIGHT_HIP', 'LEFT_HIP', 'SPINE'],
            # 'SPINE': ['THORAX'],
            # 'THORAX': ['NECK', 'LEFT_SHOULDER', 'RIGHT_SHOULDER'],
            # 'NECK': ['NOSE_ROOT'],
            # NOSE_ROOT is an end point
            'NOSE_ROOT': ['LEFT_SHOULDER', 'RIGHT_SHOULDER'],
            'LEFT_SHOULDER': ['LEFT_ELBOW'],
            'LEFT_ELBOW': ['LEFT_WRIST'],
            'LEFT_WRIST': ['LEFT_HAND_WRIST'],
            'LEFT_HAND_WRIST': ['LEFT_THUMB_MCP', 'LEFT_INDEX_FINGER_MCP', 'LEFT_MIDDLE_FINGER_MCP', 'LEFT_RING_FINGER_MCP', 'LEFT_PINKY_MCP'],

            'LEFT_THUMB_MCP': ['LEFT_THUMB_PIP'],
            'LEFT_THUMB_PIP': ['LEFT_THUMB_DIP'],
            'LEFT_THUMB_DIP': ['LEFT_THUMB_TIP_ENDPOINT'],
            'LEFT_THUMB_TIP_ENDPOINT': [],

            'LEFT_INDEX_FINGER_MCP': ['LEFT_INDEX_FINGER_PIP'],
            'LEFT_INDEX_FINGER_PIP': ['LEFT_INDEX_FINGER_DIP'],
            'LEFT_INDEX_FINGER_DIP': ['LEFT_INDEX_FINGER_TIP_ENDPOINT'],
            'LEFT_INDEX_FINGER_TIP_ENDPOINT': [],

            'LEFT_MIDDLE_FINGER_MCP': ['LEFT_MIDDLE_FINGER_PIP'],
            'LEFT_MIDDLE_FINGER_PIP': ['LEFT_MIDDLE_FINGER_DIP'],
            'LEFT_MIDDLE_FINGER_DIP': ['LEFT_MIDDLE_FINGER_TIP_ENDPOINT'],
            'LEFT_MIDDLE_FINGER_TIP_ENDPOINT': [],

            'LEFT_RING_FINGER_MCP': ['LEFT_RING_FINGER_PIP'],
            'LEFT_RING_FINGER_PIP': ['LEFT_RING_FINGER_DIP'],
            'LEFT_RING_FINGER_DIP': ['LEFT_RING_FINGER_TIP_ENDPOINT'],
            'LEFT_RING_FINGER_TIP_ENDPOINT': [],

            'LEFT_RING_FINGER_MCP': ['LEFT_RING_FINGER_PIP'],
            'LEFT_RING_FINGER_PIP': ['LEFT_RING_FINGER_DIP'],
            'LEFT_RING_FINGER_DIP': ['LEFT_RING_FINGER_TIP_ENDPOINT'],
            'LEFT_RING_FINGER_TIP_ENDPOINT': [],

            'LEFT_PINKY_MCP': ['LEFT_PINKY_PIP'],
            'LEFT_PINKY_PIP': ['LEFT_PINKY_DIP'],
            'LEFT_PINKY_DIP': ['LEFT_PINKY_TIP_ENDPOINT'],
            'LEFT_PINKY_TIP_ENDPOINT': [],

            'RIGHT_SHOULDER': ['RIGHT_ELBOW'],
            'RIGHT_ELBOW': ['RIGHT_WRIST'],
            'RIGHT_WRIST': ['RIGHT_HAND_WRIST'],
            'RIGHT_HAND_WRIST': ['RIGHT_THUMB_MCP', 'RIGHT_INDEX_FINGER_MCP', 'RIGHT_MIDDLE_FINGER_MCP', 'RIGHT_RING_FINGER_MCP', 'RIGHT_PINKY_MCP'],
            'RIGHT_THUMB_MCP': ['RIGHT_THUMB_PIP'],
            'RIGHT_THUMB_PIP': ['RIGHT_THUMB_DIP'],
            'RIGHT_THUMB_DIP': ['RIGHT_THUMB_TIP_ENDPOINT'],
            'RIGHT_THUMB_TIP_ENDPOINT': [],
            'RIGHT_INDEX_FINGER_MCP': ['RIGHT_INDEX_FINGER_PIP'],
            'RIGHT_INDEX_FINGER_PIP': ['RIGHT_INDEX_FINGER_DIP'],
            'RIGHT_INDEX_FINGER_DIP': ['RIGHT_INDEX_FINGER_TIP_ENDPOINT'],
            'RIGHT_INDEX_FINGER_TIP_ENDPOINT': [],
            'RIGHT_MIDDLE_FINGER_MCP': ['RIGHT_MIDDLE_FINGER_PIP'],
            'RIGHT_MIDDLE_FINGER_PIP': ['RIGHT_MIDDLE_FINGER_DIP'],
            'RIGHT_MIDDLE_FINGER_DIP': ['RIGHT_MIDDLE_FINGER_TIP_ENDPOINT'],
            'RIGHT_MIDDLE_FINGER_TIP_ENDPOINT': [],
            'RIGHT_RING_FINGER_MCP': ['RIGHT_RING_FINGER_PIP'],
            'RIGHT_RING_FINGER_PIP': ['RIGHT_RING_FINGER_DIP'],
            'RIGHT_RING_FINGER_DIP': ['RIGHT_RING_FINGER_TIP_ENDPOINT'],
            'RIGHT_RING_FINGER_TIP_ENDPOINT': [],
            'RIGHT_RING_FINGER_MCP': ['RIGHT_RING_FINGER_PIP'],
            'RIGHT_RING_FINGER_PIP': ['RIGHT_RING_FINGER_DIP'],
            'RIGHT_RING_FINGER_DIP': ['RIGHT_RING_FINGER_TIP_ENDPOINT'],
            'RIGHT_RING_FINGER_TIP_ENDPOINT': [],
            'RIGHT_PINKY_MCP': ['RIGHT_PINKY_PIP'],
            'RIGHT_PINKY_PIP': ['RIGHT_PINKY_DIP'],
            'RIGHT_PINKY_DIP': ['RIGHT_PINKY_TIP_ENDPOINT'],
            'RIGHT_PINKY_TIP_ENDPOINT': [],
        }
        self.parent = {self.root: None}
        for parent, children in self.children.items():
            for child in children:
                self.parent[child] = parent

        self.left_joints = [
            joint for joint in self.keypoint2index
            if 'LEFT' in joint
        ]
        self.right_joints = [
            joint for joint in self.keypoint2index
            if 'RIGHT' in joint
        ]

        hand_points_connections = [
            ("WRIST", "THUMB_MCP"),
            ("THUMB_MCP", "THUMB_IP"),
            ("THUMB_IP", "THUMB_TIP_ENDPOINT"),

            ("WRIST", "INDEX_FINGER_MCP"),
            ("INDEX_FINGER_MCP", "INDEX_FINGER_PIP"),
            ("INDEX_FINGER_PIP", "INDEX_FINGER_DIP"),
            ("INDEX_FINGER_DIP", "INDEX_FINGER_TIP_ENDPOINT"),

            ("WRIST", "MIDDLE_FINGER_MCP"),
            ("MIDDLE_FINGER_MCP", "MIDDLE_FINGER_PIP"),
            ("MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP"),
            ("MIDDLE_FINGER_DIP", "MIDDLE_FINGER_TIP_ENDPOINT"),

            ("WRIST", "RING_FINGER_MCP"),
            ("RING_FINGER_MCP", "RING_FINGER_PIP"),
            ("RING_FINGER_PIP", "RING_FINGER_DIP"),
            ("RING_FINGER_DIP", "RING_FINGER_TIP_ENDPOINT"),

            ("WRIST", "PINKY_MCP"),
            ("PINKY_MCP", "PINKY_PIP"),
            ("PINKY_PIP", "PINKY_DIP"),
            ("PINKY_DIP", "PINKY_TIP_ENDPOINT")
        ]
        # Single bones for face
        body_points_single_bones = [
            # right eye
            "RIGHT_EYE",
            # left eye
            "LEFT_EYE"
        ]
        # T-pose
        self.initial_directions = {
            # 'HIP': [0, 0, 0],
            # 'RIGHT_HIP': [-1, 0, 0],
            # 'LEFT_HIP': [1, 0, 0],
            # 'SPINE': [0, 0, 1],
            # 'THORAX': [0, 0, 1],
            # 'NECK': [0, 0, 1],
            'NOSE_ROOT': [0, 0, 0],
            'LEFT_SHOULDER': [1, 0, 0],
            'LEFT_ELBOW': [1, 0, 0],
            'LEFT_WRIST': [1, 0, 0],
            'LEFT_HAND_WRIST': [1, 0, 0],

            'LEFT_THUMB_MCP': [1, 0, 0],
            'LEFT_THUMB_PIP': [1, 0, 0],
            'LEFT_THUMB_DIP': [1, 0, 0],
            'LEFT_THUMB_TIP_ENDPOINT': [0, 0, 1],

            'LEFT_INDEX_FINGER_MCP': [1, 0, 0],
            'LEFT_INDEX_FINGER_PIP': [1, 0, 0],
            'LEFT_INDEX_FINGER_DIP': [1, 0, 0],
            'LEFT_INDEX_FINGER_TIP_ENDPOINT': [0, 0, 1],

            'LEFT_MIDDLE_FINGER_MCP': [1, 0, 0],
            'LEFT_MIDDLE_FINGER_PIP': [1, 0, 0],
            'LEFT_MIDDLE_FINGER_DIP': [1, 0, 0],
            'LEFT_MIDDLE_FINGER_TIP_ENDPOINT': [0, 0, 1],

            'LEFT_RING_FINGER_MCP': [1, 0, 0],
            'LEFT_RING_FINGER_PIP': [1, 0, 0],
            'LEFT_RING_FINGER_DIP': [1, 0, 0],
            'LEFT_RING_FINGER_TIP_ENDPOINT': [0, 0, 1],

            'LEFT_RING_FINGER_MCP': [1, 0, 0],
            'LEFT_RING_FINGER_PIP': [1, 0, 0],
            'LEFT_RING_FINGER_DIP': [1, 0, 0],
            'LEFT_RING_FINGER_TIP_ENDPOINT': [0, 0, 1],

            'LEFT_PINKY_MCP': [1, 0, 0],
            'LEFT_PINKY_PIP': [1, 0, 0],
            'LEFT_PINKY_DIP': [1, 0, 0],
            'LEFT_PINKY_TIP_ENDPOINT': [0, 0, 1],


            'RIGHT_SHOULDER': [-1, 0, 0],
            'RIGHT_ELBOW': [-1, 0, 0],
            'RIGHT_WRIST': [-1, 0, 0],
            'RIGHT_HAND_WRIST': [-1, 0, 0],
            'RIGHT_THUMB_MCP': [-1, 0, 0],
            'RIGHT_THUMB_PIP': [-1, 0, 0],
            'RIGHT_THUMB_DIP': [-1, 0, 0],
            'RIGHT_THUMB_TIP_ENDPOINT': [0, 0, 1],
            'RIGHT_INDEX_FINGER_MCP': [-1, 0, 0],
            'RIGHT_INDEX_FINGER_PIP': [-1, 0, 0],
            'RIGHT_INDEX_FINGER_DIP': [-1, 0, 0],
            'RIGHT_INDEX_FINGER_TIP_ENDPOINT': [0, 0, 1],
            'RIGHT_MIDDLE_FINGER_MCP': [-1, 0, 0],
            'RIGHT_MIDDLE_FINGER_PIP': [-1, 0, 0],
            'RIGHT_MIDDLE_FINGER_DIP': [-1, 0, 0],
            'RIGHT_MIDDLE_FINGER_TIP_ENDPOINT': [0, 0, 1],
            'RIGHT_RING_FINGER_MCP': [-1, 0, 0],
            'RIGHT_RING_FINGER_PIP': [-1, 0, 0],
            'RIGHT_RING_FINGER_DIP': [-1, 0, 0],
            'RIGHT_RING_FINGER_TIP_ENDPOINT': [0, 0, 1],
            'RIGHT_RING_FINGER_MCP': [-1, 0, 0],
            'RIGHT_RING_FINGER_PIP': [-1, 0, 0],
            'RIGHT_RING_FINGER_DIP': [-1, 0, 0],
            'RIGHT_RING_FINGER_TIP_ENDPOINT': [0, 0, 1],
            'RIGHT_PINKY_MCP': [-1, 0, 0],
            'RIGHT_PINKY_PIP': [-1, 0, 0],
            'RIGHT_PINKY_DIP': [-1, 0, 0],
            'RIGHT_PINKY_TIP_ENDPOINT': [0, 0, 1],
        }
    #############################################################

    #############################################################
    def get_initial_offset(self, poses_3d):
        bone_lens = {self.root: [0]}
        stack = [self.root]
        while stack:
            parent = stack.pop()
            p_idx = self.keypoint2index[parent]
            for child in self.children[parent]:
                if 'ENDPOINT' in child:
                    bone_lens[child] = 0.4 * bone_lens[parent]
                    continue
                stack.append(child)

                c_idx = self.keypoint2index[child]
                bone_lens[child] = np.linalg.norm(
                    poses_3d[:, p_idx] - poses_3d[:, c_idx],
                    axis=1
                )

        bone_len = {}
        for joint in bone_lens:
            if 'LEFT' in joint or 'RIGHT' in joint:
                base_name = joint.replace('LEFT', '').replace('RIGHT', '')
                left_len = np.mean(bone_lens['LEFT' + base_name])
                right_len = np.mean(bone_lens['RIGHT' + base_name])
                bone_len[joint] = (left_len + right_len) / 2
            else:
                bone_len[joint] = np.mean(bone_lens[joint])

        initial_offset = {}
        for joint, direction in self.initial_directions.items():
            direction = np.array(direction) / \
                max(np.linalg.norm(direction), 1e-12)
            initial_offset[joint] = direction * bone_len[joint]

        return initial_offset

    def get_bvh_header(self, poses_3d):
        initial_offset = self.get_initial_offset(poses_3d)

        nodes = {}
        for joint in initial_offset:
            is_root = joint == self.root
            is_end_site = 'ENDPOINT' in joint
            nodes[joint] = BvhNode(
                name=joint,
                offset=initial_offset[joint],
                rotation_order='zxy' if not is_end_site else '',
                is_root=is_root,
                is_end_site=is_end_site,
            )
        for joint, children in self.children.items():
            nodes[joint].children = [nodes[child] for child in children]
            for child in children:
                nodes[child].parent = nodes[joint]

        header = BvhHeader(root=nodes[self.root], nodes=nodes)
        return header

    @staticmethod
    def quat_dot(q0, q1):
        original_shape = q0.shape
        q0 = np.reshape(q0, [-1, 4])
        q1 = np.reshape(q1, [-1, 4])

        w0, x0, y0, z0 = q0[:, 0], q0[:, 1], q0[:, 2], q0[:, 3]
        w1, x1, y1, z1 = q1[:, 0], q1[:, 1], q1[:, 2], q1[:, 3]
        q_product = w0 * w1 + x1 * x1 + y0 * y1 + z0 * z1
        q_product = np.expand_dims(q_product, axis=1)
        q_product = np.tile(q_product, [1, 4])

        return np.reshape(q_product, original_shape)

    @staticmethod
    def quat_inverse(q):
        original_shape = q.shape
        q = np.reshape(q, [-1, 4])

        q_conj = [q[:, 0], -q[:, 1], -q[:, 2], -q[:, 3]]
        q_conj = np.stack(q_conj, axis=1)
        q_inv = np.divide(q_conj, TokenLSC.quat_dot(q_conj, q_conj))

        return np.reshape(q_inv, original_shape)

    @staticmethod
    def quat_mul(q0, q1):
        original_shape = q0.shape
        q1 = np.reshape(q1, [-1, 4, 1])
        q0 = np.reshape(q0, [-1, 1, 4])
        terms = np.matmul(q1, q0)
        w = terms[:, 0, 0] - terms[:, 1, 1] - terms[:, 2, 2] - terms[:, 3, 3]
        x = terms[:, 0, 1] + terms[:, 1, 0] - terms[:, 2, 3] + terms[:, 3, 2]
        y = terms[:, 0, 2] + terms[:, 1, 3] + terms[:, 2, 0] - terms[:, 3, 1]
        z = terms[:, 0, 3] - terms[:, 1, 2] + terms[:, 2, 1] + terms[:, 3, 0]

        q_product = np.stack([w, x, y, z], axis=1)
        return np.reshape(q_product, original_shape)

    @staticmethod
    def quat_divide(q, r):
        return TokenLSC.quat_mul(TokenLSC.quat_inverse(r), q)

    @staticmethod
    def quat2euler(q, order='zxy', eps=1e-8):
        original_shape = list(q.shape)
        original_shape[-1] = 3
        q = np.reshape(q, [-1, 4])

        q0 = q[:, 0]
        q1 = q[:, 1]
        q2 = q[:, 2]
        q3 = q[:, 3]

        if order == 'zxy':
            x = np.arcsin(np.clip(2 * (q0 * q1 + q2 * q3), -1 + eps, 1 - eps))
            y = np.arctan2(2 * (q0 * q2 - q1 * q3),
                           1 - 2 * (q1 * q1 + q2 * q2))
            z = np.arctan2(2 * (q0 * q3 - q1 * q2),
                           1 - 2 * (q1 * q1 + q3 * q3))
            euler = np.stack([z, x, y], axis=1)
        else:
            raise ValueError('Not implemented')

        return np.reshape(euler, original_shape)

    @staticmethod
    def norm(x):
        return x / max(np.linalg.norm(x), 1e-12)

    @staticmethod
    def dcm_from_axis(x_dir, y_dir, z_dir, order):
        assert order in ['yzx', 'yxz', 'xyz', 'xzy', 'zxy', 'zyx']

        axis = {'x': x_dir, 'y': y_dir, 'z': z_dir}
        name = ['x', 'y', 'z']
        idx0 = name.index(order[0])
        idx1 = name.index(order[1])
        idx2 = name.index(order[2])

        axis[order[0]] = TokenLSC.norm(axis[order[0]])
        axis[order[1]] = TokenLSC.norm(np.cross(
            axis[name[(idx1 + 1) % 3]], axis[name[(idx1 + 2) % 3]]
        ))
        axis[order[2]] = TokenLSC.norm(np.cross(
            axis[name[(idx2 + 1) % 3]], axis[name[(idx2 + 2) % 3]]
        ))

        dcm = np.asarray([axis['x'], axis['y'], axis['z']])

        return dcm

    @staticmethod
    def dcm2quat(dcm):
        q = np.zeros([4])
        tr = np.trace(dcm)

        if tr > 0:
            sqtrp1 = np.sqrt(tr + 1.0)
            q[0] = 0.5 * sqtrp1
            q[1] = (dcm[1, 2] - dcm[2, 1]) / (2.0 * sqtrp1)
            q[2] = (dcm[2, 0] - dcm[0, 2]) / (2.0 * sqtrp1)
            q[3] = (dcm[0, 1] - dcm[1, 0]) / (2.0 * sqtrp1)
        else:
            d = np.diag(dcm)
            if d[1] > d[0] and d[1] > d[2]:
                sqdip1 = np.sqrt(d[1] - d[0] - d[2] + 1.0)
                q[2] = 0.5 * sqdip1

                if sqdip1 != 0:
                    sqdip1 = 0.5 / sqdip1

                q[0] = (dcm[2, 0] - dcm[0, 2]) * sqdip1
                q[1] = (dcm[0, 1] + dcm[1, 0]) * sqdip1
                q[3] = (dcm[1, 2] + dcm[2, 1]) * sqdip1

            elif d[2] > d[0]:
                sqdip1 = np.sqrt(d[2] - d[0] - d[1] + 1.0)
                q[3] = 0.5 * sqdip1

                if sqdip1 != 0:
                    sqdip1 = 0.5 / sqdip1

                q[0] = (dcm[0, 1] - dcm[1, 0]) * sqdip1
                q[1] = (dcm[2, 0] + dcm[0, 2]) * sqdip1
                q[2] = (dcm[1, 2] + dcm[2, 1]) * sqdip1

            else:
                sqdip1 = np.sqrt(d[0] - d[1] - d[2] + 1.0)
                q[1] = 0.5 * sqdip1

                if sqdip1 != 0:
                    sqdip1 = 0.5 / sqdip1

                q[0] = (dcm[1, 2] - dcm[2, 1]) * sqdip1
                q[2] = (dcm[0, 1] + dcm[1, 0]) * sqdip1
                q[3] = (dcm[2, 0] + dcm[0, 2]) * sqdip1

        return q

    def pose2euler(self, pose, header):
        channel = []
        quats = {}
        eulers = {}
        stack = [header.root]
        while stack:
            node = stack.pop()
            joint = node.name
            joint_idx = self.keypoint2index[joint]

            if node.is_root:
                channel.extend(pose[joint_idx])

            index = self.keypoint2index
            order = None
            if joint == 'NOSE_ROOT':
                x_dir = pose[index['LEFT_HIP']] - pose[index['RIGHT_HIP']]
                y_dir = None
                z_dir = pose[joint_idx] - \
                    (pose[index['LEFT_HIP']] - pose[index['RIGHT_HIP']])
                order = 'zyx'
            elif joint == 'LEFT_SHOULDER':
                x_dir = pose[index['LEFT_ELBOW']] - pose[joint_idx]
                y_dir = pose[index['LEFT_ELBOW']] - pose[index['LEFT_WRIST']]
                z_dir = None
                order = 'xzy'
            elif joint == 'LEFT_ELBOW':
                x_dir = pose[index['LEFT_WRIST']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_SHOULDER']]
                z_dir = None
                order = 'xzy'
            elif joint == 'LEFT_WRIST':
                x_dir = pose[index['LEFT_HAND_WRIST']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_ELBOW']]
                z_dir = None
                order = 'xzy'
            elif joint == 'LEFT_HAND_WRIST':
                x_dir = pose[index['LEFT_THUMB_MCP']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_WRIST']]
                z_dir = None
                order = 'xzy'
            elif joint == 'LEFT_THUMB_MCP':
                x_dir = pose[index['LEFT_THUMB_PIP']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_HAND_WRIST']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'LEFT_THUMB_PIP':
                x_dir = pose[index['LEFT_THUMB_DIP']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_THUMB_MCP']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'LEFT_THUMB_DIP':
                x_dir = pose[index['LEFT_THUMB_TIP_ENDPOINT']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_THUMB_PIP']]
                z_dir = None
                order = 'xzy'

            elif joint == 'LEFT_INDEX_FINGER_MCP':
                x_dir = pose[index['LEFT_INDEX_FINGER_PIP']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_HAND_WRIST']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'LEFT_INDEX_FINGER_PIP':
                x_dir = pose[index['LEFT_INDEX_FINGER_DIP']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_INDEX_FINGER_MCP']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'LEFT_INDEX_FINGER_DIP':
                x_dir = pose[index['LEFT_INDEX_FINGER_TIP_ENDPOINT']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_INDEX_FINGER_PIP']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'LEFT_MIDDLE_FINGER_MCP':
                x_dir = pose[index['LEFT_MIDDLE_FINGER_PIP']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_HAND_WRIST']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'LEFT_MIDDLE_FINGER_PIP':
                x_dir = pose[index['LEFT_MIDDLE_FINGER_DIP']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_MIDDLE_FINGER_MCP']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'LEFT_MIDDLE_FINGER_DIP':
                x_dir = pose[index['LEFT_MIDDLE_FINGER_TIP_ENDPOINT']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_MIDDLE_FINGER_PIP']]
                z_dir = None
                order = 'xzy'
            elif joint == 'LEFT_RING_FINGER_MCP':
                x_dir = pose[index['LEFT_RING_FINGER_PIP']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_HAND_WRIST']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'LEFT_RING_FINGER_PIP':
                x_dir = pose[index['LEFT_RING_FINGER_DIP']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_RING_FINGER_MCP']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'LEFT_RING_FINGER_DIP':
                x_dir = pose[index['LEFT_RING_FINGER_TIP_ENDPOINT']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_RING_FINGER_PIP']]
                z_dir = None
                order = 'xzy'

            elif joint == 'LEFT_PINKY_MCP':
                x_dir = pose[index['LEFT_PINKY_PIP']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_HAND_WRIST']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'LEFT_PINKY_PIP':
                x_dir = pose[index['LEFT_PINKY_DIP']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_PINKY_MCP']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'LEFT_PINKY_DIP':
                x_dir = pose[index['LEFT_PINKY_TIP_ENDPOINT']] - pose[joint_idx]
                y_dir = pose[joint_idx] - pose[index['LEFT_PINKY_PIP']]
                z_dir = None
                order = 'xzy'

            

            elif joint == 'RIGHT_SHOULDER':
                x_dir = pose[joint_idx] - pose[index['RIGHT_ELBOW']]
                y_dir = pose[index['RIGHT_ELBOW']] - pose[index['RIGHT_WRIST']]
                z_dir = None
                order = 'xzy'
            elif joint == 'RIGHT_ELBOW':
                x_dir = pose[joint_idx] - pose[index['RIGHT_WRIST']]
                y_dir = pose[joint_idx] - pose[index['RIGHT_SHOULDER']]
                z_dir = None
                order = 'xzy'
            elif joint == 'RIGHT_WRIST':
                x_dir =  pose[joint_idx] - pose[index['RIGHT_HAND_WRIST']]
                y_dir = pose[joint_idx] - pose[index['LEFT_ELBOW']]
                z_dir = None
            elif joint == 'RIGHT_HAND_WRIST':
                x_dir =  pose[joint_idx] - pose[index['LEFT_MIDDLE_FINGER_MCP']]
                y_dir = pose[joint_idx] - pose[index['RIGHT_WRIST']]
                z_dir = None
                order = 'xzy'
            elif joint == 'RIGHT_THUMB_MCP':
                x_dir = pose[joint_idx] - pose[index['RIGHT_THUMB_PIP']] 
                y_dir = pose[joint_idx] - pose[index['RIGHT_HAND_WRIST']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'RIGHT_THUMB_PIP':
                x_dir =  pose[joint_idx]-pose[index['RIGHT_THUMB_DIP']]
                y_dir = pose[joint_idx] - pose[index['RIGHT_THUMB_MCP']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'RIGHT_THUMB_DIP':
                x_dir =  pose[joint_idx] - pose[index['RIGHT_THUMB_TIP_ENDPOINT']]
                y_dir = pose[joint_idx] - pose[index['RIGHT_THUMB_PIP']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'RIGHT_INDEX_FINGER_MCP':
                x_dir =  pose[joint_idx] - pose[index['RIGHT_INDEX_FINGER_PIP']]
                y_dir = pose[joint_idx] - pose[index['RIGHT_HAND_WRIST']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'RIGHT_INDEX_FINGER_PIP':
                x_dir = pose[joint_idx] - pose[index['RIGHT_INDEX_FINGER_DIP']]
                y_dir = pose[joint_idx] - pose[index['RIGHT_INDEX_FINGER_MCP']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'RIGHT_INDEX_FINGER_DIP':
                x_dir =  pose[joint_idx] - pose[index['RIGHT_INDEX_FINGER_TIP_ENDPOINT']]
                y_dir = pose[joint_idx] - pose[index['RIGHT_INDEX_FINGER_PIP']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'RIGHT_MIDDLE_FINGER_MCP':
                x_dir =  pose[joint_idx] - pose[index['RIGHT_MIDDLE_FINGER_PIP']]
                y_dir = pose[joint_idx] - pose[index['RIGHT_HAND_WRIST']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'RIGHT_MIDDLE_FINGER_PIP':
                x_dir =  pose[joint_idx] - pose[index['RIGHT_MIDDLE_FINGER_DIP']]
                y_dir = pose[joint_idx] - pose[index['RIGHT_MIDDLE_FINGER_MCP']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'RIGHT_MIDDLE_FINGER_DIP':
                x_dir =  pose[joint_idx] - pose[index['RIGHT_MIDDLE_FINGER_TIP_ENDPOINT']]
                y_dir = pose[joint_idx] - pose[index['RIGHT_MIDDLE_FINGER_PIP']]
                z_dir = None
                order = 'xzy'
            elif joint == 'RIGHT_RING_FINGER_MCP':
                x_dir = pose[joint_idx] - pose[index['RIGHT_RING_FINGER_PIP']]
                y_dir = pose[joint_idx] - pose[index['RIGHT_HAND_WRIST']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'RIGHT_RING_FINGER_PIP':
                x_dir =  pose[joint_idx]-pose[index['RIGHT_RING_FINGER_DIP']] 
                y_dir = pose[joint_idx] - pose[index['RIGHT_RING_FINGER_MCP']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'RIGHT_RING_FINGER_DIP':
                x_dir =  pose[joint_idx] - pose[index['RIGHT_RING_FINGER_TIP_ENDPOINT']]
                y_dir = pose[joint_idx] - pose[index['RIGHT_RING_FINGER_PIP']]
                z_dir = None
                order = 'xzy'

            elif joint == 'RIGHT_PINKY_MCP':
                x_dir =  pose[joint_idx] - pose[index['RIGHT_PINKY_PIP']]
                y_dir = pose[joint_idx] - pose[index['RIGHT_HAND_WRIST']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'RIGHT_PINKY_PIP':
                x_dir =  pose[joint_idx] - pose[index['RIGHT_PINKY_DIP']]
                y_dir = pose[joint_idx] - pose[index['RIGHT_PINKY_MCP']]
                z_dir = None
                order = 'xzy'
            
            elif joint == 'RIGHT_PINKY_DIP':
                x_dir =  pose[joint_idx] - pose[index['RIGHT_PINKY_TIP_ENDPOINT']]
                y_dir = pose[joint_idx] - pose[index['RIGHT_PINKY_PIP']]
                z_dir = None
                order = 'xzy'




            if order:
                dcm = TokenLSC.dcm_from_axis(x_dir, y_dir, z_dir, order)
                quats[joint] = TokenLSC.dcm2quat(dcm)
            else:
                quats[joint] = quats[self.parent[joint]].copy()

            local_quat = quats[joint].copy()
            if node.parent:
                local_quat = TokenLSC.quat_divide(
                    q=quats[joint], r=quats[node.parent.name]
                )

            euler = TokenLSC.quat2euler(
                q=local_quat, order=node.rotation_order
            )
            euler = np.rad2deg(euler)
            eulers[joint] = euler
            channel.extend(euler)

            for child in node.children[::-1]:
                if not child.is_end_site:
                    stack.append(child)

        return channel

    def export_to_bvh(self, header=None, output_file=None):
        if not header:
            header = self.get_bvh_header(self.poses3d)

        channels = []
        for frame, pose in enumerate(self.poses3d):
            channels.append(self.pose2euler(pose, header))

        if output_file:
            write_bvh(output_file, header, channels)

        return channels, header

    def render_bvh(self, path: str):
        pass

    def export_to_avatar_video(self, path: str):
        pass

    def export_to_human_video(self, path: str):
        pass
##########################################################################################################################

    @staticmethod
    def interpolate(sign1: List[Skeleton], sign2: List[Skeleton], prev_cut=2, next_cut=2, amount_of_skeletons=4, kind='cubic'):

        origin_skeletons = sign1[-prev_cut:]

        end_skeletons = sign2[:next_cut]

        l = prev_cut + next_cut + amount_of_skeletons

        dims = origin_skeletons[0].all_points.shape

        newbodies = np.zeros((amount_of_skeletons, dims[0], dims[1]))

        fullbodies = [sk.all_points for sk in origin_skeletons+end_skeletons]

        for i, points in enumerate(zip(*fullbodies)):

            current_kind = kind

            x = [point[0] for point in points]

            y = [point[1] for point in points]

            z = [point[2] for point in points]

            t = [j for j in range(prev_cut)] + [prev_cut +
                                                amount_of_skeletons + j for j in range(next_cut)]

            if len(t) < 4:
                current_kind = "linear"

            fx = interp1d(t, x, kind=current_kind)
            fy = interp1d(t, y, kind=current_kind)
            fz = interp1d(t, z, kind=current_kind)

            newx = [fx(p)
                    for p in range(prev_cut, prev_cut + amount_of_skeletons)]
            newy = [fy(p)
                    for p in range(prev_cut, prev_cut + amount_of_skeletons)]
            newz = [fz(p)
                    for p in range(prev_cut, prev_cut + amount_of_skeletons)]

            for j in range(amount_of_skeletons):
                X, Y, Z = (newx[j], newy[j], newz[j])
                newbodies[j, i, 0] = X
                newbodies[j, i, 1] = Y
                newbodies[j, i, 2] = Z

        skeleton_list = []

        body_dim = origin_skeletons[0].body.shape[0]
        lhand_dim = origin_skeletons[0].lhand.shape[0]
        rhand_dim = origin_skeletons[0].rhand.shape[0]

        for i in range(amount_of_skeletons):
            body = newbodies[i, :body_dim, :]

            lhand = newbodies[i, body_dim:body_dim+lhand_dim, :]

            rhand = newbodies[i, body_dim +
                              lhand_dim:body_dim+lhand_dim+rhand_dim, :]

            body_part = Bodypart("body", TypeOfBodyPart.BODY, body)
            left_part = Bodypart("left", TypeOfBodyPart.LHAND, lhand)
            right_part = Bodypart("right", TypeOfBodyPart.RHAND, rhand)
            skeleton = Skeleton(body_part, left_part, right_part)
            skeleton_list.append(skeleton)

        return skeleton_list

    @staticmethod
    def join_tokens(token1, token2, prev_cut=2, next_cut=2, amount_of_skeletons=None, kind='linear'):

        sign1, sign2 = token1.sign, token2.sign
        new_name = f"{token1.name}-{token2.name}"

        last_sk = sign1[-1].all_points
        first_sk = sign2[0].all_points

        if amount_of_skeletons is None:
            difference = np.linalg.norm(last_sk - first_sk)
            f = interp1d([0, 0.5, 4.7, 9], [0, 8, 14, 18],
                         assume_sorted=True, kind="linear", fill_value="extrapolate")
            amount_of_skeletons = int(np.ceil(f(difference)))
        elif amount_of_skeletons < 0:
            raise ValueError("amount_of_skeleton is negative")

        list_skeleton = token1.interpolate(sign1, sign2,
                                           prev_cut=prev_cut,
                                           next_cut=next_cut,
                                           amount_of_skeletons=amount_of_skeletons,
                                           kind=kind)

        new_sign = sign1+list_skeleton+sign2

        return TokenLSC(new_name, new_sign, with_normalize=False, with_crop=False)

    def __add__(self, object):
        '''
        Interpolate here with two LSC tokens
        '''

        if not isinstance(object, TokenLSC):
            raise ValueError(
                f"Invalid type {type(object)} in addition with TokenLSC")

        return self.join_tokens(self, object)

    def normalize(self, rhand_threshold=2, lhand_threshold=2, previous_cut=2, next_cut=2, edge_interpolation_gap=5, hand_forearm_proportion=3/11, finger_proportion=1/0.68):

        if self.isNormalized:
            pass

        self.isNormalize = True

        for i, sk in enumerate(self.sign):
            body, lhand, rhand = sk.bodyparts
            body.normalize()
            lhand.normalize()
            rhand.normalize()
            sk.normalize()

        rhand_mask = [np.linalg.norm(
            x.rhand.points) > rhand_threshold for x in self.sign]
        lhand_mask = [np.linalg.norm(
            x.lhand.points) > lhand_threshold for x in self.sign]

        def get_to_fix_intervals(mask):
            to_fix = []
            i = 0
            while i < len(mask):
                if not mask[i]:
                    false_start = i
                    while i < len(mask) and not mask[i]:
                        i += 1
                    false_end = i
                    to_fix.append((false_start, false_end))
                else:
                    i += 1
            return to_fix

        def fix_interval(intervals: list, body_part, mask: list):

            def get_default_pos(part: TypeOfBodyPart, frame: int):
                sk = self.sign[frame]
                if part == TypeOfBodyPart.LHAND:
                    
                    return sk.get_position_default_hand(True, hand_forearm_proportion, finger_proportion)
                elif part == TypeOfBodyPart.RHAND:
                   
                    return sk.get_position_default_hand(False, hand_forearm_proportion, finger_proportion)

            def fix_ends(start, end):
                true_list = []
                for i in range(start, end):
                    default = get_default_pos(body_part, i)
                    bodypart = None
                    if body_part == TypeOfBodyPart.BODY:
                        bodypart = self.sign[i].body
                    elif body_part == TypeOfBodyPart.LHAND:
                        bodypart = self.sign[i].lhand
                    elif body_part == TypeOfBodyPart.RHAND:
                        bodypart = self.sign[i].rhand
                    else:
                        raise Exception(f"Invalid body part. {body_part}")
                    true_list.append(True)
                    bodypart.points = default
                mask[start:end] = true_list

            interpolation_gap = edge_interpolation_gap

            if intervals and intervals[0][0] == 0:
                # Starts with an invalid position
                start, end = intervals.pop(0)
                gap_size = end - start
                if gap_size > interpolation_gap:
                    fix_ends(start, end - interpolation_gap)
                    intervals.insert(0, (end - interpolation_gap, end))
                elif gap_size == 1:
                    fix_ends(start, end)
                else:
                    middle = gap_size // 2
                    fix_ends(start, start + middle)
                    intervals.insert(0, (end - middle, end))
            if intervals and intervals[-1][1] == len(mask):
                # Ends with an invalid position
                start, end = intervals.pop()
                fix_ends(start, end)
                gap_size = end - start
                if gap_size > interpolation_gap:
                    fix_ends(start + interpolation_gap, end)
                    intervals.append((start, start + interpolation_gap))
                elif gap_size == 1:
                    fix_ends(start, end)
                else:
                    middle = gap_size // 2
                    fix_ends(start + middle, end)
                    intervals.append((start, start + middle))

            for false_start, false_end in intervals:
                left = self.sign[:false_start]
                right = self.sign[false_end:]
                amount = false_end - false_start

                # Extend previous cut backward
                current_previous_cut = 1
                while false_start-current_previous_cut > 0 and mask[false_start-current_previous_cut] and current_previous_cut <= previous_cut:
                    current_previous_cut += 1
                current_previous_cut -= 1

                # Extend next cut forward
                current_next_cut = 1
                while false_end + current_next_cut < len(mask) and mask[false_end + current_next_cut] and current_next_cut < next_cut:
                    current_next_cut += 1

                gap_signs = self.interpolate(
                    left, right, prev_cut=current_previous_cut, next_cut=current_next_cut, amount_of_skeletons=amount)

                for sk, fixed in zip(self.sign[false_start:false_end], gap_signs):
                    if body_part == TypeOfBodyPart.BODY:
                        sk.body = fixed.body
                    elif body_part == TypeOfBodyPart.LHAND:
                        sk.lhand = fixed.lhand
                    elif body_part == TypeOfBodyPart.RHAND:
                        sk.rhand = fixed.rhand

        rhand_intervals = get_to_fix_intervals(rhand_mask)
        lhand_intervals = get_to_fix_intervals(lhand_mask)

        fix_interval(rhand_intervals, TypeOfBodyPart.RHAND, rhand_mask)
        fix_interval(lhand_intervals, TypeOfBodyPart.LHAND, lhand_mask)

        angle = None
        for sk in self.sign:
            angle = self._scale_normalize(sk, angle=angle)

    def _scale_normalize(self, sk, angle=None):
        
        # control_points = [24, 23] # right hip, left hip
        control_points = [12, 11] # right shoulder, left shoulder
        
        r_control, l_control = sk.body.points[control_points]
        control_vector = l_control - r_control
        norm = np.linalg.norm(control_vector)
        
        factor = 0.5 / norm
        
        for part in sk.bodyparts:
            part.points = factor * part.points
        
        # Rotate to align XY plane
        # x_vector = np.array([1,0,0])
        # axis = cross_product(control_vector, x_vector)
        # angle = get_angle_between_vectors(control_vector, x_vector) if angle is None else angle
        # for part in sk.bodyparts:
        #     part.points = np.array(
        #         [rodriquez_rotation_formula(v, angle, axis) for v in part.points])
        # return angle

    def crop_movement(self,
                      movement_percentage=.8,
                      focus_on: TypeOfBodyPart = None,
                      difference_mapping=lambda x: np.linalg.norm(x) ** 2,
                      to_maximize=lambda value, i, j: value / abs(j-i),
                      integration_method="simpson"):
        """
        integration_method: trapezoid or simpson
        """
        if self.isCropped:
            pass

        self.isCropped = True

        if focus_on is None:
            general = [sk.all_points for sk in self.sign]
            general = [general[i] - general[i+1]
                       for i in range(len(general) - 1)]
            to_focus = general
        elif focus_on == TypeOfBodyPart.BODY:
            rhands = [sk.rhand for sk in self.sign]
            rhands = [rhands[i].points -
                      rhands[i+1].points for i in range(len(rhands) - 1)]
            to_focus = rhands
        elif focus_on == TypeOfBodyPart.LHAND:
            lhands = [sk.lhand for sk in self.sign]
            lhands = [lhands[i].points -
                      lhands[i+1].points for i in range(len(lhands) - 1)]
            to_focus = lhands
        elif focus_on == TypeOfBodyPart.RHAND:
            body = [sk.body for sk in self.sign]
            body = [body[i].points -
                    body[i+1].points for i in range(len(body) - 1)]
            to_focus = body
        else:
            raise ValueError("Invalid focus_on:", focus_on)

        def integrate(samples, a, b):
            """
            a: lower integration limit
            b: upper integration limit
            """
            from scipy import integrate

            X = [i for i in range(a, b+1)]
            Y = samples[a:b+1]

            if integration_method == "simpson":
                value = integrate.simpson(Y, X)
            elif integration_method == "trapezoid":
                value = integrate.trapezoid(Y, X)
            else:
                raise ValueError(f"Invalid value: {integration_method}")
            return value

        # Integrate the squared norm
        integrate_with = [difference_mapping(x) for x in to_focus]

        integration_values = {}
        maximization_values = {}
        total_movement = 0

        for i in range(len(integrate_with)-1):
            for j in range(i+1, len(integrate_with)):
                value = integrate(integrate_with, i, j)
                if j == i + 1:
                    total_movement += value
                integration_values[i, j] = value
                maximization_values[i, j] = to_maximize(value, i, j)

        i_max = 0
        j_max = 0
        value_max = 0
        # Can add some restrictions if necessary
        for i in range(len(integrate_with)-1):
            for j in range(i+1, len(integrate_with)):
                int_val = integration_values[i, j]
                max_val = maximization_values[i, j]
                current_movement_percentage = int_val / total_movement
                if max_val > value_max and current_movement_percentage >= movement_percentage:
                    value_max = max_val
                    i_max = i
                    j_max = j

        self.sign = self.sign[i_max:j_max+1]

        return i_max, j_max

    def animate(self, fps=30, figsize=(7.0, 3.5), elev=90, angle=1, dims=None, text=False, joints=True):
        if not dims:
            dims = self.dims

        skeletons_points = np.array([sk.all_points for sk in self.sign])
        rc('animation', html='jshtml')
        fig_size_x, fig_size_y = figsize

        plt.rcParams["figure.figsize"] = [fig_size_x, fig_size_y]
        plt.rcParams['figure.autolayout'] = True

        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        colors = ['b', 'y', 'r', 'k']

        total_frames = skeletons_points.shape[0]
        time = total_frames / fps

        def update(i):
            ax.clear()

            D = self.__transform_data(skeletons_points, i)

            s = 0
            e = 0
            for j, dim in enumerate(dims):
                dim = dim[0]
                e += dim

                x = D[s:e, 0]
                y = D[s:e, 1]
                z = D[s:e, 2]

                ax.plot(D[s:e, 0], D[s:e, 1], D[s:e, 2], f'{colors[j]}.')
                if text:
                    for p in range(s, e):
                        ax.text3D(D[p, 0], D[p, 1], D[p, 2], str(p))
                if joints:
                    for h, k in self.fullbodyjoints[j]:
                        ax.plot([D[s+h, 0], D[s+k, 0]], [D[s+h, 1], D[s+k, 1]],
                                [D[s+h, 2], D[s+k, 2]], f'{colors[j]}')
                s += dim

                ax.set_axis_off()
                if elev and angle:
                    ax.view_init(elev, angle)

            return ax

        return FuncAnimation(fig, update, frames=total_frames, repeat=True, interval=fps)

    def __transform_data(self, Data, i):
        from scipy.spatial.transform import Rotation as R

        r = R.from_rotvec(np.pi/2 * np.array([0, 0, 1]))

        X = np.array(Data[i, :, 0])
        Y = np.array(-Data[i, :, 1])
        Z = np.array(Data[i, :, 2])

        D = np.array([x for x in zip(X, Y, Z)])
        D = r.apply(D)
        return D

    def __str__(self) -> str:
        return f"Token: {self.name}. Frames: {len(self.sign)}"

    def __repr__(self) -> str:
        return str(self)

    @staticmethod
    def _get_images_from_video(filename, folder=None, fps=25, limit=None):

        filepath = os.path.join(folder, filename) if folder else filename
        cap = cv2.VideoCapture(filepath)
        if fps:
            _fps = math.ceil(cap.get(cv2.CAP_PROP_FPS))
            step = math.floor(_fps / fps)
            frame_index = -1
        images = []
        try:
            while cap.isOpened():
                success, image = cap.read()
                if not success:
                    break
                if fps:
                    frame_index += 1
                    if frame_index % step:
                        continue
                images.append(image)
                if limit is not None and len(images) >= limit:
                    break
        finally:
            cap.release()
            cv2.destroyAllWindows()
        return images

    @staticmethod
    def _get_landmarks(array, shape):
        array = np.array([[item.x, item.y, item.z] for item in array])
        landmarks = np.zeros(shape)
        if not array.size:
            return landmarks
        landmarks[: array.shape[0], : array.shape[1]] = array
        return landmarks

    @staticmethod
    def _get_pose_landmarks(r):
        landmarks = []
        if r and r.pose_landmarks and r.pose_landmarks.landmark:
            landmarks = r.pose_landmarks.landmark
        return TokenLSC._get_landmarks(landmarks, (33, 3))

    @staticmethod
    def _get_left_hand_landmarks(r):
        landmarks = []
        if r and r.left_hand_landmarks and r.left_hand_landmarks.landmark:
            landmarks = r.left_hand_landmarks.landmark
        return TokenLSC._get_landmarks(landmarks, (21, 3))

    @staticmethod
    def _get_right_hand_landmarks(r):
        landmarks = []
        if r and r.right_hand_landmarks and r.right_hand_landmarks.landmark:
            landmarks = r.right_hand_landmarks.landmark
        return TokenLSC._get_landmarks(landmarks, (21, 3))

    @staticmethod
    def _process_results(results) -> List[Skeleton]:
        pose = TokenLSC._get_pose_landmarks
        left = TokenLSC._get_left_hand_landmarks
        right = TokenLSC._get_right_hand_landmarks
        skeletons = []
        for r in results:
            p = pose(r)[:25, :]
            l = left(r)
            r = right(r)
            body_part = Bodypart("body", TypeOfBodyPart.BODY, p)
            left_part = Bodypart("left", TypeOfBodyPart.LHAND, l)
            right_part = Bodypart("right", TypeOfBodyPart.RHAND, r)
            skeleton = Skeleton(body_part, left_part, right_part)
            skeletons.append(skeleton)

        return skeletons

    @staticmethod
    def import_token_lsc_from_path(path: str, name: str = '_new-sign_', verbose=False, with_normalize=True, with_crop=True) -> 'TokenLSC':
        if '.json' in path:
            return TokenLSC.import_token_lsc_from_json(str(path), with_normalize=True, with_crop=True)
        else:
            return TokenLSC.import_token_lsc_from_video(path, name, with_normalize=True, with_crop=True)

    @staticmethod
    def import_token_lsc_from_video(path: str, name: str = '_new-sign_', verbose=False, with_normalize=True, with_crop=True) -> 'TokenLSC':
        images = TokenLSC._get_images_from_video(path)

        with mp.solutions.holistic.Holistic() as holistic:
            results = []
            for image in images:
                result = holistic.process(image)
                results.append(result)
            sign = TokenLSC._process_results(results)

            return TokenLSC(name, sign, with_normalize=with_normalize, with_crop=with_crop)

    @staticmethod
    def import_token_lsc_from_json(path: str, name: str = '', verbose=False, with_normalize=True, with_crop=True) -> 'TokenLSC':
        json_path = Path(path)

        json_list = json.loads(json_path.read_text())

        if len(json_list) > 1:
            print(
                f"More than one representation found at {path}. Selecting first representation.")

        json_repr = json_list[0]

        np_repr = np.array(json_repr)
        if verbose:
            print("Json shape:", np_repr.shape)

        np_repr = np.reshape(
            np_repr, (np_repr.shape[0], np_repr.shape[1] // 3, 3))
        if verbose:
            print("Frames shape", np_repr.shape)

        body = np_repr[:, :25, :]
        left = np_repr[:, 25:46, :]
        right = np_repr[:, 46:, :]

        if verbose:
            print("Body shape", body.shape)
            print("Left shape", left.shape)
            print("Right shape", right.shape)

        skeletons = []
        for frame_body, frame_left, frame_right in zip(body, left, right):
            body_part = Bodypart("body", TypeOfBodyPart.BODY, frame_body)
            left_part = Bodypart("left", TypeOfBodyPart.LHAND, frame_left)
            right_part = Bodypart("right", TypeOfBodyPart.RHAND, frame_right)
            skeleton = Skeleton(body_part, left_part, right_part)
            skeletons.append(skeleton)

        if name == '':
            name = json_path.name.split(".")[0]
        return TokenLSC(name, skeletons, with_normalize=with_normalize, with_crop=with_crop)


In [ ]:
def plot_difference_token_lsc(token: TokenLSC, threshold=None):

    rhands = [sk.rhand for sk in token.sign]
    lhands = [sk.lhand for sk in token.sign]
    body = [sk.body for sk in token.sign]

    def plot(title, X, Y, label="Norma del vector", ylabel="Valor", xlabel="Tiempo", threshold=None):
        plt.plot(X, Y, label=label)
        if threshold is not None:
            plt.plot(X, [threshold for _ in X], label="Umbral")
        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.legend()
        plt.show()

    X = list(range(len(rhands)))

    for title, ann_list in [
        ("Diferencia de mano derecha", rhands),
        ("Diferencia de mano izquierda", lhands),
        ("Diferencia de cuerpo", body),
    ]:
        plot(title, X, [np.linalg.norm(x.points) for x in ann_list])


def movement_plot(token: TokenLSC, threshold=None, i_max=None, j_max=None):

    rhands = [sk.rhand for sk in token.sign]
    rhands = [rhands[i].points -
              rhands[i+1].points for i in range(len(rhands) - 1)]
    lhands = [sk.lhand for sk in token.sign]
    lhands = [lhands[i].points -
              lhands[i+1].points for i in range(len(lhands) - 1)]
    body = [sk.body for sk in token.sign]
    body = [body[i].points - body[i+1].points for i in range(len(body) - 1)]
    general = [sk.all_points for sk in token.sign]
    general = [general[i] - general[i+1] for i in range(len(general) - 1)]

    def plot(title, X, Y, label="Norma del vector diferencia", ylabel="Valor", xlabel="Tiempo"):
        max_value = max(Y)

        plt.plot(X, Y, label=label)

        if threshold is not None:
            plt.plot(X, [threshold for _ in X], label="Umbral")
        if i_max is not None:
            plt.plot([i_max, i_max], [0, max_value])
        if j_max is not None:
            plt.plot([j_max, j_max], [0, max_value])

        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.legend()
        plt.show()

    X = list(range(len(rhands)))

    for title, ann_list in [
        ("Movimiento de mano derecha", rhands),
        ("Movimiento de mano izquierda", lhands),
        ("Movimiento del cuerpo", body),
            ("Movimiento general", general),]:
        plot(title, X, [np.linalg.norm(x) ** 2 for x in ann_list])


def analyze_token_lsc(json_path: str, with_normalize=True, with_crop=True):
    tokenlsc = TokenLSC.import_token_lsc_from_json(
        json_path, with_normalize=with_normalize, with_crop=with_crop)
    tokenlsc_normalized = TokenLSC.import_token_lsc_from_json(json_path)
    tokenlsc_crop = TokenLSC.import_token_lsc_from_json(json_path)

    print("Animacion sin procesamiento")

    print("Se muestran glitches en las manos")
    print("Detecctar los glitches con un umbral con su distancia al 0 de las coordenadas")
    plot_difference_token_lsc(tokenlsc)

    print("Interpolar los puntos faltantes con los existentes")
    tokenlsc_normalized.normalize()
    tokenlsc_crop.normalize()

    plot_difference_token_lsc(tokenlsc_normalized)

    print("La seña posee un período de inactividad grande, no necesario para la comunicación")
    i_max, j_max = tokenlsc_crop.crop_movement()
    print("Se seleccionan los frames comprendidos entre ", i_max, "y",
          j_max, "los cuales contienen la mayor cantidad de movimiento")
    movement_plot(tokenlsc_normalized, i_max=None, j_max=None)
    movement_plot(tokenlsc_crop)

    return tokenlsc, tokenlsc_normalized, tokenlsc_crop


In [ ]:
# normal, normalized, cropped = analyze_token_lsc(
#     "./jsons/amar.json", with_normalize=False, with_crop=False)
# # normal, normalized, cropped = analyze_token_lsc("abandonar.json")

# Problemas encontrados, Manos Glitch, Otra cosa ??
# Plotear plot_difference_token_lsc. con el umbral => Manos Glitch
# ... Otros problemas
# Propuesta de solucion

# Normalizar
# Plotear plot_difference_token_lsc. con el umbral, para que se vean que se arreglaron los problemas
# Animacion con normalizado

# Problemas encontrados: De inmovilidad de avatar, otra cosa??
# Ploteas movement_plot => inmovilidad de avatar
# ... Otros problemas
# Propuesta de solucion

# Experimentacion
# PROBARLO CON OTROS TOKENS. Para ajustar hiperparametros
# Union de tokens
# Union de tokens procesados


In [ ]:
# normal.animate(joints=True)


In [ ]:
# normal.sign[0].plot()


In [ ]:
# normal.sign[-1].plot()


In [ ]:
# normalized.animate(joints=True)


In [ ]:
# cropped.animate(joints=True)


In [ ]:
def multiple_tokens_plot(json_tokens: List[str]):
    final_token = None
    for json_path in json_tokens:
        token = TokenLSC.import_token_lsc_from_json(json_path)
        if final_token is None:
            final_token = token
        else:
            final_token += token

    return final_token.animate(joints=True), final_token


In [ ]:
a, token = multiple_tokens_plot(
    [
        # "jsons/abandonar.json",
        "jsons/amar.json",
        # "jsons/amar.json",
        "jsons/aborto.json",
    ])
a


In [ ]:
X = []
Y1 = []
Y2 = []
Y3 = []
Y4 = []
for i, sk in enumerate(token.sign):
    X.append(i)
    # y1 = sk.body.points[12] - sk.body.points[24]
    # y2 = sk.body.points[11] - sk.body.points[23]
    # y3 = sk.body.points[12] - sk.body.points[11]
    ax = np.array([0,0,1])
    y1 = sk.body.points[12] * ax 
    y2 = sk.body.points[11] * ax
    y3 = sk.body.points[23] * ax
    y4 = sk.body.points[24] * ax

    Y1.append(np.linalg.norm(y1))
    Y2.append(np.linalg.norm(y2))
    Y3.append(np.linalg.norm(y3))
    Y4.append(np.linalg.norm(y4))
plt.plot(X, Y1, label="r s")
plt.plot(X, Y2, label="l s")
plt.plot(X, Y3, label="l h")
plt.plot(X, Y4, label="r h")
plt.legend()
plt.show()

# a.save('./videos_matplotlib/abandonar+aborto+amar+amar.mp4')


In [ ]:
amar = TokenLSC.import_token_lsc_from_video(
    "./videos_input/AMAR.mp4", "amar", with_normalize=False, with_crop=False)
# amar = TokenLSC.import_token_lsc_from_json("amar.json")
amar.normalize()
amar.crop_movement()
# amar.animate()


In [ ]:
# amar.animate(joints=True)


In [ ]:
new_sign = amar+amar
# animation = new_sign.animate(joints=True)
# animation


In [ ]:
b=new_sign.export_to_bvh(output_file='a.bvh')

In [ ]:
# animation.save('test_abandonar_amar.mp4')


In [ ]:
# dataset=None
# with open("./dataset.json", 'r', encoding='utf-8') as json_list:
#       dataset=json.load(json_list)
#       json_list.close()


In [ ]:
# with open('./dataset2.json', 'w', encoding='utf-8') as json_file:
#   json.dump(dataset, json_file, sort_keys=True, ensure_ascii=False)
#   json_file.close()


In [ ]:
# with open('./bombon.json', 'w', encoding='utf-8') as json_file:
#   json.dump(dataset["bombon"], json_file, indent=4,
#             sort_keys=True, ensure_ascii=False)
#   json_file.close()


In [ ]:
# with open('./aborto.json', 'w', encoding='utf-8') as json_file:
#   json.dump(dataset["aborto"], json_file, indent=4,
#             sort_keys=True, ensure_ascii=False)
#   json_file.close()


In [ ]:
# import json
# import numpy as np
# with open("./aborto.json", 'r', encoding='utf-8') as json_list:
#     aborto_json = json.load(json_list)
